In [ ]:
from functions import *
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture
from tqdm import tqdm
import math
import cmath
from scipy.integrate import simpson, romb
from scipy.stats import iqr, norm, gaussian_kde
from scipy.special import erfc, wofz
import pickle
import os
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
theta_list, x_list = np.loadtxt("DataPhase28.dat", unpack=True)
T = len(theta_list)
alpha = 0.3
r2 = 0.925

L_limit = 4
n_precision = 10000
X_precision = np.linspace(-L_limit, L_limit, n_precision)
N_theta = 28

In [ ]:
grouped_theta_list, grouped_data_dict = data_wrapper(theta_list, x_list, N_theta)

### with $\mu$ and fixed $\nu$

In [ ]:
mu_max = 8
nu = 1
kappa = 0.8
N_mu = 100
gmm_components = 3

mu_list = np.linspace(0, mu_max, N_mu)


phi = np.arctan(nu/mu_list)
theta_list_from_mu = np.where(phi<=0, phi, phi + np.pi)


eps = 1e-12
cos_th = np.cos(theta_list_from_mu)
sin_th = np.sin(theta_list_from_mu)
r_list = np.where(np.abs(cos_th) > eps, mu_list / cos_th, nu / sin_th)

# store one solution per mu
solutions_mu_nu = list(zip(mu_list, r_list, theta_list_from_mu))

true_phi_list = []
estimated_gaussian_list = []
corrected_gaussian_list = []
theta_bin_list = []


for mu, radius, theta in tqdm(solutions_mu_nu):
    true_phi_list.append(phi_1(theta, alpha, r2, radius, t=1))
    theta_bin = int(np.floor(np.mod(theta, np.pi) / (np.pi / N_theta)))
    xb = np.array(grouped_data_dict[theta_bin])
    W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
    phi_t1_vals_gaussian = compute_t_fourier_transform(W_Gaussian, X_precision, t=radius)
    estimated_gaussian_list.append(phi_t1_vals_gaussian)
    theta_bin_list.append(theta_bin)
    
    # phi_t1_vals_gaussian = compute_t_fourier_transform(W_Gaussian, X_precision, t=radius / kappa)
    # phi_t1_vals_gaussian *= 1 / np.exp( - ((1 - kappa) / kappa * radius)**2 / 2 )
    # corrected_gaussian_list.append(phi_t1_vals_gaussian)

In [ ]:
plt.plot(mu_list, np.array(true_phi_list).real, label='True $Re[\phi(t=1)]$', color = 'blue')
plt.plot(mu_list, np.array(true_phi_list).imag, label='True $Im[\phi(t=1)]$', color = 'orange')

plt.plot(mu_list, np.array(estimated_gaussian_list).real, '--', label='Estimated $Re[\phi(t=1)]$', color = 'blue')
plt.plot(mu_list, np.array(estimated_gaussian_list).imag, '--', label='Estimated $Im[\phi(t=1)]$', color = 'orange')

# plt.plot(mu_list, np.array(corrected_gaussian_list).real, '--.', label='Corrected $Re[\phi_1]$', color = 'blue')
# plt.plot(mu_list, np.array(corrected_gaussian_list).imag, '--.', label='Corrected $Im[\phi_1]$', color = 'orange')

plt.legend()
plt.xlabel(r"$\mu$")
plt.ylabel(r"$\phi(1, \mu, \nu)$")
plt.title(rf"$\phi(1,\mu,\nu)$, $\nu$ = {nu}")

In [ ]:
theta_target = 0.24

mu_max = 8
nu = 1
kappa = 1
N_mu = 100


mu_list = np.linspace(-mu_max, mu_max, N_mu)

mu_target = nu / np.tan(theta_target)
print(f"Target mu for theta={theta_target}: {mu_target}")

mu_target = 4

index_target = (np.abs(mu_list - mu_target)).argmin()
print(f"Index of target mu: {index_target}, mu value: {mu_list[index_target]}")

In [ ]:
# Generate the plot

In [ ]:
# Read data from pickle file
with open('lvovsky_KCFE.pkl', 'rb') as f:
    data = pickle.load(f)
    KCFE_estimated_gaussian_list = data['estimated_gaussian_list']
    KCFE_corrected_gaussian_list = data['corrected_gaussian_list']

In [ ]:
# make global linewidth the same for each line
linewidth = 2



phi = np.arctan(nu/mu_list)
theta_list_from_mu = np.where(phi<=0, phi, phi + np.pi)


eps = 1e-12
cos_th = np.cos(theta_list_from_mu)
sin_th = np.sin(theta_list_from_mu)
r_list = np.where(np.abs(cos_th) > eps, mu_list / cos_th, nu / sin_th)

# store one solution per mu
solutions_mu_nu = list(zip(mu_list, r_list, theta_list_from_mu))

gmm_components = 1

true_phi_list = []
estimated_gaussian_list = []
corrected_gaussian_list = []
theta_bin_list = []


for mu, radius, theta in tqdm(solutions_mu_nu):
    true_phi_list.append(phi_1(theta, alpha, r2, radius, t=1))
    theta_bin = int(np.floor(np.mod(theta, np.pi) / (np.pi / N_theta)))
    xb = np.array(grouped_data_dict[theta_bin])
    W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
    phi_t1_vals_gaussian = compute_t_fourier_transform(W_Gaussian, X_precision, t=radius)
    estimated_gaussian_list.append(phi_t1_vals_gaussian)
    theta_bin_list.append(theta_bin)

plt.plot(mu_list, np.array(true_phi_list).real, label='Theoretical $Re[\phi]$', color = 'red', linewidth=linewidth)
plt.plot(mu_list, np.array(true_phi_list).imag, '--', label='Theoretical $Im[\phi]$', color = 'red', linewidth=linewidth)

plt.plot(mu_list, np.array(estimated_gaussian_list).real, label='MLE $Re[\hat{\phi}]$ ($GMM=$1)', color = 'orange', linewidth=linewidth)
plt.plot(mu_list, np.array(estimated_gaussian_list).imag, '--', label='MLE $Im[\hat{\phi}]$ ($GMM=$1)', color = 'orange', linewidth=linewidth)

print('SE for MLE (GMM=1) at mu_target:', np.abs(np.array(estimated_gaussian_list)[index_target] - np.array(true_phi_list)[index_target])**2)
print('total mean SE for MLE (GMM=1):', np.mean(np.abs(np.array(estimated_gaussian_list) - np.array(true_phi_list))**2))

gmm_components = 2

true_phi_list = []
estimated_gaussian_list = []
corrected_gaussian_list = []
theta_bin_list = []


for mu, radius, theta in tqdm(solutions_mu_nu):
    true_phi_list.append(phi_1(theta, alpha, r2, radius, t=1))
    theta_bin = int(np.floor(np.mod(theta, np.pi) / (np.pi / N_theta)))
    xb = np.array(grouped_data_dict[theta_bin])
    W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
    phi_t1_vals_gaussian = compute_t_fourier_transform(W_Gaussian, X_precision, t=radius)
    estimated_gaussian_list.append(phi_t1_vals_gaussian)
    theta_bin_list.append(theta_bin)


plt.plot(mu_list, np.array(estimated_gaussian_list).real, label='MLE $Re[\hat{\phi}]$ ($GMM=$2)', color = 'blue', linewidth=linewidth)
plt.plot(mu_list, np.array(estimated_gaussian_list).imag, '--', label='MLE $Im[\hat{\phi}]$ ($GMM=$2)', color = 'blue', linewidth=linewidth)

print('SE for MLE (GMM=2) at mu_target:', np.abs(np.array(estimated_gaussian_list)[index_target] - np.array(true_phi_list)[index_target])**2)
print('total mean SE for MLE (GMM=2):', np.mean(np.abs(np.array(estimated_gaussian_list) - np.array(true_phi_list))**2))

plt.plot(mu_list, np.array(KCFE_estimated_gaussian_list).real, label='Gaussian KCFE $Re[\hat{\phi}]$', color = 'green', linewidth=linewidth)
plt.plot(mu_list, np.array(KCFE_estimated_gaussian_list).imag, '--', label='Gaussian KCFE $Im[\hat{\phi}]$', color = 'green', linewidth=linewidth)

print('SE for KCFE at mu_target:', np.abs(np.array(KCFE_estimated_gaussian_list)[index_target] - np.array(true_phi_list)[index_target])**2)
print('total mean SE for KCFE:', np.mean(np.abs(np.array(KCFE_estimated_gaussian_list) - np.array(true_phi_list))**2))

plt.legend(ncols=2)
plt.xlabel(r"$\mu$")
plt.ylabel(r"$\hat{\phi}(t=1)$")
plt.title(rf"$\phi(t=1)$ estimation")

plt.tight_layout()
plt.savefig('Lvovsky_KCFE.pdf')


In [ ]:
# # make global linewidth the same for each line
# linewidth = 2

# mu_max = 8
# nu = 1
# kappa = 0.9
# N_mu = 100


# mu_list = np.linspace(0, mu_max, N_mu)


# phi = np.arctan(nu/mu_list)
# theta_list_from_mu = np.where(phi<=0, phi, phi + np.pi)


# eps = 1e-12
# cos_th = np.cos(theta_list_from_mu)
# sin_th = np.sin(theta_list_from_mu)
# r_list = np.where(np.abs(cos_th) > eps, mu_list / cos_th, nu / sin_th)

# # store one solution per mu
# solutions_mu_nu = list(zip(mu_list, r_list, theta_list_from_mu))

# gmm_components = 1

# true_phi_list = []
# estimated_gaussian_list = []
# corrected_gaussian_list = []
# theta_bin_list = []


# for mu, radius, theta in tqdm(solutions_mu_nu):
#     true_phi_list.append(phi_1(theta, alpha, r2, radius, t=1))
#     theta_bin = int(np.floor(np.mod(theta, np.pi) / (np.pi / N_theta)))
#     xb = np.array(grouped_data_dict[theta_bin])
#     W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
#     phi_t1_vals_gaussian = compute_t_fourier_transform(W_Gaussian, X_precision, t=radius / kappa)
#     phi_t1_vals_gaussian *= 1 / np.exp( - ((1 - kappa) / kappa * radius)**2 / 2 )
#     corrected_gaussian_list.append(phi_t1_vals_gaussian)

# plt.plot(mu_list, np.array(true_phi_list).real, label='Theoretical $Re[\phi]$', color = 'red', linewidth=linewidth)
# plt.plot(mu_list, np.array(true_phi_list).imag, '--', label='Theoretical $Im[\phi]$', color = 'red', linewidth=linewidth)

# plt.plot(mu_list, np.array(corrected_gaussian_list).real, label='MLE $Re[\hat{\phi}]$ ($GMM=$1)', color = 'orange', linewidth=linewidth)
# plt.plot(mu_list, np.array(corrected_gaussian_list).imag, '--', label='MLE $Im[\hat{\phi}]$ ($GMM=$1)', color = 'orange', linewidth=linewidth)


# print('SE for MLE (GMM=1) at mu_target:', np.abs(np.array(corrected_gaussian_list)[index_target] - np.array(true_phi_list)[index_target])**2)

# gmm_components = 2

# true_phi_list = []
# estimated_gaussian_list = []
# corrected_gaussian_list = []
# theta_bin_list = []


# for mu, radius, theta in tqdm(solutions_mu_nu):
#     true_phi_list.append(phi_1(theta, alpha, r2, radius, t=1))
#     theta_bin = int(np.floor(np.mod(theta, np.pi) / (np.pi / N_theta)))
#     xb = np.array(grouped_data_dict[theta_bin])
#     W_Gaussian = reconstruct_pdf_mle(xb, X_precision, gmm_components)
#     phi_t1_vals_gaussian = compute_t_fourier_transform(W_Gaussian, X_precision, t=radius / kappa)
#     phi_t1_vals_gaussian *= 1 / np.exp( - ((1 - kappa) / kappa * radius)**2 / 2 )
#     corrected_gaussian_list.append(phi_t1_vals_gaussian)



# plt.plot(mu_list, np.array(corrected_gaussian_list).real, label='MLE $Re[\hat{\phi}]$ ($GMM=$2)', color = 'blue', linewidth=linewidth)
# plt.plot(mu_list, np.array(corrected_gaussian_list).imag, '--', label='MLE $Im[\hat{\phi}]$ ($GMM=$2)', color = 'blue', linewidth=linewidth)

# print('SE for MLE (GMM=2) at mu_target:', np.abs(np.array(corrected_gaussian_list)[index_target] - np.array(true_phi_list)[index_target])**2)

# plt.plot(mu_list, np.array(KCFE_corrected_gaussian_list).real, label='Gaussian KCFE $Re[\hat{\phi}]$', color = 'green', linewidth=linewidth)
# plt.plot(mu_list, np.array(KCFE_corrected_gaussian_list).imag, '--', label='Gaussian KCFE $Im[\hat{\phi}]$', color = 'green', linewidth=linewidth)

# print('SE for KCFE at mu_target:', np.abs(np.array(KCFE_corrected_gaussian_list)[index_target] - np.array(true_phi_list)[index_target])**2)


# plt.legend(ncol=2)
# plt.xlabel(r"$\mu$")
# plt.ylabel(r"$\hat{\phi}(t=1)$")
# plt.title(rf"$\phi(t=1)$ estimation with Gaussian noise inversion, $\kappa$ = {kappa}")

# plt.tight_layout()
# plt.savefig('Lvovsky_KCFE_correction.pdf')